In [3]:
import pyupbit
import numpy as np
df = pyupbit.get_ohlcv("KRW-BTC", count=7)
print(df)

                           open        high         low       close  \
2023-01-19 09:00:00  25701000.0  26299000.0  25690000.0  26205000.0   
2023-01-20 09:00:00  26208000.0  28105000.0  26101000.0  28026000.0   
2023-01-21 09:00:00  28025000.0  29000000.0  27721000.0  28333000.0   
2023-01-22 09:00:00  28333000.0  28706000.0  27937000.0  28390000.0   
2023-01-23 09:00:00  28400000.0  28890000.0  28220000.0  28609000.0   
2023-01-24 09:00:00  28600000.0  28951000.0  28014000.0  28351000.0   
2023-01-25 09:00:00  28363000.0  28428000.0  28001000.0  28357000.0   

                          volume         value  
2023-01-19 09:00:00  3265.225511  8.473315e+10  
2023-01-20 09:00:00  6019.113075  1.617842e+11  
2023-01-21 09:00:00  7165.779761  2.036514e+11  
2023-01-22 09:00:00  4350.789211  1.237055e+11  
2023-01-23 09:00:00  4463.876856  1.273641e+11  
2023-01-24 09:00:00  6049.392006  1.732244e+11  
2023-01-25 09:00:00  1568.997463  4.431398e+10  


In [13]:
import pyupbit
import numpy as np

class backTesting :
    def __init__(self, daily_data, start_cash) :
        self.daily_data = daily_data # 일봉 데이터
        self.fee = 0.0011 # 수수료 ( calculate_fee.xlsx 참고 )
        self.buy_signal = False # 매수 신호
        
        self.start_cash = start_cash # 시작 자산
        self.current_cash = start_cash # 현재 자산
        self.highest_cash = start_cash # 자산 최고점
        self.lowest_cash = start_cash # 자산 최저점

        self.ror = 1 # 수익률
        self.accumulated_ror = 1 # 누적 수익률
        self.mdd = 0 # 최대 낙폭

        self.trade_count = 0 # 거래횟수
        self.win_count = 0 # 승리횟수

    def execute(self) :

        # 노이즈 계산 ( 1- 절대값(시가 - 종가) / (고가 - 저가) )
        self.daily_data['noise'] = 1 - abs(self.daily_data['open'] - self.daily_data['close']) / (self.daily_data['high'] - self.daily_data['low'])
        # 노이즈 20일 평균
        self.daily_data['noise_ma20'] = self.daily_data['noise'].rolling(window=20, min_periods=1).mean()

        # 변동폭 ( 고가 - 저가 )
        self.daily_data['range'] = self.daily_data['high'] - self.daily_data['low']
        # 목표매수가 ( 시가 + 변동폭 * K )
        self.daily_data['targetPrice'] = self.daily_data['open'] + self.daily_data['range'].shift(1) * self.daily_data['noise_ma20']
        # 5일 이동평균선
        self.daily_data['ma5'] = self.daily_data['close'].rolling(window=5, min_periods=1).mean().shift(1)
        # 상승장 여부
        self.daily_data['bull'] = self.daily_data['open'] > self.daily_data['ma5']

        for idx, row in df.iterrows() :
            # 매수 신호 확인
            self.buy_signal = np.where((row['targetPrice'] <= row['high']) & row['bull'], True, False) # 목표가에 달성한 경우 목표가에 매수해 다음날 시가에 매도한 것으로 판단 + 상승장 조건 추가

            # 거래횟수 계산
            self.trade_count += 1 if self.buy_signal else 0

            # 수익률 계산
            self.ror = row['close'] / row['targetPrice'] - self.fee if self.buy_signal else 1 # 다음날 시가와 당일 종가의 시간차이가 거의 없으므로 종가로 계산

            # 승리 횟수 계산
            self.win_count += 1 if self.ror > 1 else 0

            # 누적 수익률 계산
            self.accumulated_ror *= self.ror
            
            # 현재 자산 갱신
            self.current_cash *= self.ror

            # 자산 최고점 갱신
            self.highest_cash = max(self.highest_cash, self.current_cash)

            # 자산 최저점 갱신
            self.lowest_cash = min(self.lowest_cash, self.current_cash)

            # 최대 낙폭 계산
            dd = (self.highest_cash - self.current_cash) / self.highest_cash * 100
            self.mdd = max(self.mdd, dd)

        self.result()

    def result(self) :
        print('='*40)
        print('테스트 결과')
        print('-'*40)
        print('총 거래 횟수 : %s' %self.trade_count)
        print('승리 횟수 : %s' %self.win_count)
        print('승률 : %s' %(self.win_count / self.trade_count * 100))
        print('누적 수익률 : %s' %self.accumulated_ror)
        print('현재 잔액 : %s' % self.current_cash)
        print('최고 잔액 : %s' % self.highest_cash)
        print('최저 잔액 : %s' % self.lowest_cash)
        print('최대 낙폭 (MDD) : %s' % self.mdd)
        print('='*40)

df = pyupbit.get_ohlcv("KRW-BTC", count=365) # 일봉 데이터
backtest = backTesting(df, 1000000)
backtest.execute()

테스트 결과
----------------------------------------
총 거래 횟수 : 58
승리 횟수 : 25
승률 : 43.103448275862064
누적 수익률 : 0.7921362488284345
현재 잔액 : 792136.2488284349
최고 잔액 : 1089724.5215247637
최저 잔액 : 718776.7459441604
최대 낙폭 (MDD) : 34.040509161119545
